# Q1

In [42]:
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim 
import folium
location = Nominatim(user_agent="toronto_explorer").geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude

In [10]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = pd.read_html(req.text)[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [11]:
df = df[df["Borough"] != "Not assigned"]
df = df.groupby(['Postal Code']).head()
df = df.reset_index()
df.drop(['index'], axis = 'columns', inplace = True)
df.shape
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Q2

In [14]:
import geocoder

geodata = pd.read_csv("https://cocl.us/Geospatial_data")
combined_df = df.join(geodata.set_index('Postal Code'), on='Postal Code', how='inner')

In [15]:
combined_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Q3

Here we generate the nearby venues

In [17]:
CLIENT_ID = '4MBQIG0LEEVVOFV5ZL2WI2E0TGX4WD2HVPXIRFBLIROXT3WU' 
CLIENT_SECRET = 'WG0AR5QAZVX1PDSBQTOURL125ZP4LVV50W0YFH4QCIQVU3YH'

def getNearbyVenues(names, latitudes, longitudes):
    venues_list=[]
    for name, lat, lon in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            "20180323", 
            lat, 
            lon, 
            300
            )
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lon, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

venues = getNearbyVenues(names=combined_df['Neighbourhood'], latitudes=combined_df['Latitude'],longitudes=combined_df['Longitude'])
venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,Park
1,Victoria Village,43.725882,-79.315572,Victoria Village Arena,Hockey Arena
2,Victoria Village,43.725882,-79.315572,Tim Hortons,Coffee Shop
3,Victoria Village,43.725882,-79.315572,Portugril,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,The Frig,French Restaurant
...,...,...,...,...,...
845,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Healthy Planet,Supplement Shop
846,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,7-Eleven,Convenience Store
847,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,McDonald's,Fast Food Restaurant
848,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,RONA,Hardware Store


Then we get the most common venues

In [28]:
venue_cat = pd.get_dummies(venues[['Venue Category']],prefix="", prefix_sep="")
venue_cat['Neighbourhood'] = venues['Neighbourhood'] 
venue_cat = venue_cat[[venue_cat.columns[-1]] + list(venue_cat.columns[:-1])].groupby('Neighbourhood').mean().reset_index()

col = ['Neighbourhood']
for i in np.arange(10):
    col.append('{} Most Common Venue'.format(i+1))
    
neighborhoods_venue_cat= pd.DataFrame(columns=col)
neighborhoods_venue_cat['Neighbourhood'] = venue_cat['Neighbourhood']
for i in np.arange(venue_cat.shape[0]):
    neighborhoods_venue_cat.iloc[i, 1:] = venue_cat.iloc[i, :].iloc[1:].sort_values(ascending=False).index.values[0:10]

neighborhoods_venue_cat

,Neighbourhood,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Yoga Studio,Farmers Market,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pharmacy,Gym,Pizza Place,Coffee Shop,Pub,Dessert Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Mobile Phone Shop,Restaurant,Shopping Mall,Middle Eastern Restaurant,Sandwich Place,Supermarket,Gas Station,Pizza Place,Sushi Restaurant
3,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pub,Thai Restaurant,Restaurant,Juice Bar,Liquor Store,Comfort Food Restaurant
4,Berczy Park,Grocery Store,Italian Restaurant,Pub,Restaurant,Liquor Store,Breakfast Spot,Beer Bar,Concert Hall,Gay Bar,Dim Sum Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
75,Weston,Park,Jewelry Store,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop
76,"Willowdale, Willowdale West",Coffee Shop,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
77,Woburn,Korean BBQ Restaurant,Yoga Studio,Farmers Market,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
78,Woodbine Heights,Intersection,Dance Studio,Beer Store,Bar,Yoga Studio,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant


Now we build the model

In [40]:
model = KMeans(n_clusters=5, random_state=42).fit(venue_cat.drop('Neighbourhood', 1))
#neighborhoods_venue_cat.insert(0, 'Cluster Labels', model.labels_)
dup = combined_df
merged_df = dup.join(neighborhoods_venue_cat.set_index('Neighbourhood'), on='Neighbourhood').dropna(subset=['Cluster Labels'])
merged_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run,Dive Bar
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Intersection,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,French Restaurant,Yoga Studio,Escape Room,Electronics Store,Dumpling Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Park,Bakery,Coffee Shop,Design Studio,Distribution Center,Bus Stop,Breakfast Spot,Food Truck,Italian Restaurant,Gym / Fitness Center
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Furniture / Home Store,Women's Store,Coffee Shop,Event Space,Boutique,Gluten-free Restaurant,Dessert Shop,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Italian Restaurant,Café,Bubble Tea Shop,Park,Sushi Restaurant,Restaurant,Sandwich Place,Thai Restaurant,Portuguese Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,1.0,Café,Coffee Shop,Restaurant,Gym,Seafood Restaurant,Tea Room,Gym / Fitness Center,Gastropub,Japanese Restaurant,Deli / Bodega
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,1.0,Pool,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run,Dive Bar
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1.0,Coffee Shop,Breakfast Spot,Flower Shop,Ramen Restaurant,Pizza Place,Beer Bar,Ice Cream Shop,Diner,Bookstore,Ethiopian Restaurant
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,1.0,Farmers Market,Auto Workshop,Brewery,Light Rail Station,Fast Food Restaurant,Garden,Park,General Travel,Event Space,Escape Room


And now we do our map

In [48]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
for lat, lon, poi, cluster in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Neighbourhood'], merged_df['Cluster Labels']):
    folium.CircleMarker(
        [lat, lon],
        popup=folium.Popup('Cluster ' + str(int(cluster) +1)),
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters